In [9]:
import numpy as np
from scipy.stats import t

In [10]:
class BayesianLinearRegressionT:
    def __init__(self, noise_var, nu=3):
        """
        Bayesian Linear Regression with Student's t-distribution prior for robustness against outliers.
        :param noise_var: Observation noise variance (sigma_y^2)
        :param nu: Degrees of freedom for the t-distribution (lower values give heavier tails)
        """
        self.noise_var = noise_var  # Noise variance
        self.nu = nu  # Degrees of freedom for t-distribution
        self.coefficients = None  # Placeholder for the coefficients
        self.X_train = None  # To store training data for kernel-based prediction
    
    def _t_log_prior(self, theta):
        """
        Log-prior for Student's t-distribution.
        :param theta: Coefficients
        """
        # Compute the log-pdf of the t-distribution for each coefficient
        return np.sum(t.logpdf(theta, df=self.nu))
    
    def _log_likelihood(self, K, y, theta):
        """
        Log-likelihood for the data given the parameters (Gaussian likelihood).
        :param K: Kernel matrix (n_samples, n_samples)
        :param y: Target vector
        :param theta: Coefficients
        """
        y_pred = np.dot(K, theta)
        residual = y - y_pred
        log_likelihood = -0.5 * np.sum((residual ** 2) / self.noise_var)
        return log_likelihood
    
    def _log_posterior(self, K, y, theta):
        """
        Log-posterior is a combination of log-prior and log-likelihood.
        :param K: Kernel matrix (n_samples, n_samples)
        :param y: Target vector
        :param theta: Coefficients
        """
        return self._log_likelihood(K, y, theta) + self._t_log_prior(theta)
    
    def fit(self, X, y, kernel_func, max_iter=1000, lr=0.01):
        """
        Fit the model using kernel transformation and gradient ascent.
        :param X: Feature matrix
        :param y: Target vector
        :param kernel_func: Kernel transformation function (e.g., rbf_kernel, polynomial_kernel)
        :param max_iter: Maximum iterations for optimization
        :param lr: Learning rate for gradient ascent
        """
        self.X_train = X  # Store training data to compute kernel at prediction time
        
        # Apply kernel transformation to training data
        K_train = kernel_func(X)
        
        n_samples = K_train.shape[0]
        
        # Initialize coefficients randomly
        self.coefficients = np.random.randn(n_samples)
        
        # Gradient ascent to maximize the log-posterior
        for _ in range(max_iter):
            grad_log_post = self._compute_gradient(K_train, y, self.coefficients)
            self.coefficients += lr * grad_log_post
    
    def _compute_gradient(self, K, y, theta):
        """
        Numerical gradient of the log-posterior with respect to the coefficients.
        :param K: Kernel matrix (n_samples, n_samples)
        :param y: Target vector
        :param theta: Coefficients
        """
        eps = 1e-5
        grad = np.zeros_like(theta)
        
        for i in range(len(theta)):
            theta_pos = np.copy(theta)
            theta_neg = np.copy(theta)
            theta_pos[i] += eps
            theta_neg[i] -= eps
            
            log_post_pos = self._log_posterior(K, y, theta_pos)
            log_post_neg = self._log_posterior(K, y, theta_neg)
            
            grad[i] = (log_post_pos - log_post_neg) / (2 * eps)
        
        return grad
    
    def predict(self, X_new, kernel_func):
        """
        Predict using the kernel between new data and training data.
        :param X_new: New data for prediction
        :param kernel_func: Kernel transformation function
        """
        # Compute the kernel between the new data and training data
        K_new = kernel_func(X_new, self.X_train)
        
        # Predict using the kernel-transformed data and learned coefficients
        return np.dot(K_new, self.coefficients)


In [11]:
def polynomial_kernel(X, X_train=None, degree=3, coef0=1):
    """
    Polynomial kernel transformation for non-linear relationships.
    :param X: New data
    :param X_train: Training data (None if fitting the model, used for prediction)
    :param degree: Degree of the polynomial kernel
    :param coef0: Bias term
    """
    if X_train is None:
        X_train = X
    return (np.dot(X, X_train.T) + coef0) ** degree

def rbf_kernel(X, X_train=None, gamma=0.1):
    """
    RBF kernel transformation for non-linear relationships.
    :param X: New data
    :param X_train: Training data (None if fitting the model, used for prediction)
    :param gamma: Kernel coefficient
    """
    if X_train is None:
        X_train = X
    
    pairwise_sq_dists = np.sum(X ** 2, axis=1).reshape(-1, 1) + np.sum(X_train ** 2, axis=1) - 2 * np.dot(X, X_train.T)
    return np.exp(-gamma * pairwise_sq_dists)


In [35]:
# Generate synthetic data
n_samples, n_features = 100, 4
X = np.random.randn(n_samples, n_features)
true_coeff = np.array([4.0, 3.0, 2.0, -1.5])
y = np.dot(X, true_coeff) + np.random.randn(n_samples) * 0.5

# Initialize Bayesian regression model with Student's t prior and polynomial kernel
bayes_model = BayesianLinearRegressionT(noise_var=0.25, nu=3)

# Apply polynomial kernel transformation
bayes_model.fit(X, y, kernel_func=polynomial_kernel, max_iter=1000, lr=0.01)

# Generate new data for prediction
X_new = np.random.randn(10, n_features)

# Predict on the new kernel-transformed data
predictions = bayes_model.predict(X_new, kernel_func=polynomial_kernel)

print("Predictions with Polynomial Kernel and Student's t Prior:", predictions)


Predictions with Polynomial Kernel and Student's t Prior: [-4.42350856e+13  1.04970950e+14 -2.75334738e+13 -1.93868590e+13
 -1.86372537e+13  4.49501731e+13  1.41116618e+13  1.09022023e+13
  4.42650142e+12  4.25831885e+14]


In [36]:
predictions

array([-4.42350856e+13,  1.04970950e+14, -2.75334738e+13, -1.93868590e+13,
       -1.86372537e+13,  4.49501731e+13,  1.41116618e+13,  1.09022023e+13,
        4.42650142e+12,  4.25831885e+14])